# <h4 align=center> **Victoria RISO** </h4>

# <h4 align=center> **PROYECTO INDIVIDUAL Nº1 - Machine Learning Operations (MLOps)** </h4>

# <h4 align=center>**`TRANSFORMACIONES`**</h4>

In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#agregados necesarios
import ast

In [2]:
# leo los datos movies_dataset.csv
# df = pd.read_csv('data.csv')
df = pd.read_csv('movies_dataset.csv')

In [3]:
# veo nombre de las columnas, tipo de datos y cantidad de valores no nulos. 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Transformacion 1:<br />
Eliminar las columnas que no serán utilizadas: video, imdb_id, adult, original_title, poster_path y homepage.

In [4]:
# elimino las columnas indicadas

df = df.drop(['adult', 'homepage','imdb_id','original_title','poster_path', 'video'], axis=1)

Transformacion 2:<br />
Los valores nulos del campo release date deben eliminarse.

In [5]:
df['release_date'].isnull().sum()  # release_date 87 null

np.int64(87)

In [6]:
# elimino las 87 filas que tienen release_date = null
df.drop(df[df['release_date'].isna()].index, inplace=True)

Transformacion 3:<br />
Los valores nulos de los campos revenue y budget deben ser rellenados por el número 0.

In [7]:
print('Hay revenue = null:', df['revenue'].isnull().sum())  
print('Hay budget = null:', df['budget'].isnull().sum())  

Hay revenue = null: 3
Hay budget = null: 0


In [8]:
# Relleno con 0 los valores nulos de la columna revenue
# Por suerte no hay budget null. No tendré problemas al crear la columna return, salvo que haya budget en 0 (ya veré)
df['revenue'] = df['revenue'].fillna(0)

Transformación 4: </br>
Crear la columna con el retorno de inversión, llamada return dividiendo los campos revenue y budget, 
revenue / budget.</br>
Si no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [9]:
# consulto los tipos de datos de las columnas revenue y budget. Para dividirlas deberían ser numéricas 
print('Tipo de datos de la columna revenue:', df['revenue'].dtype)
print('Tipo de datos de la columna budget:', df['budget'].dtype)

Tipo de datos de la columna revenue: float64
Tipo de datos de la columna budget: object


In [10]:
# Tengo que convertir la columna budget a float64

# df1['budget'] = df1['budget'].astype(float, copy = True)
# could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'
# Ah! En una fila budget tiene ese valor! - RESOLVERLO MAS ADELANTE. NO OLVIDAR!

# pandas.to_numeric
# errors{‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
# If ‘coerce’, then invalid parsing will be set as NaN. --> y despues los completo con 0
# Budget = 0 dará return infinito. Ver cómo pasar a 0

df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].fillna(0)

In [11]:
# Creo la columna retorno de inversión
# return = revenue / budget
# Si no hay datos disponibles para calcularlo, deberá tomar el valor 0. 
df['return'] = df['revenue']/df['budget']
df['return'] = df['return'].fillna(0)

Transformación 5: </br>
De haber fechas, deberán tener el formato AAAA-mm-dd, </br>
Además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [12]:
# reviso el formato de la columna release date
df['release_date'].unique()

array(['1995-10-30', '1995-12-15', '1995-12-22', ..., '1904-03-05',
       '1991-05-13', '1917-10-21'], dtype=object)

In [47]:
# Parece que ya está en el formato pedido
# Pero cuando corro la instrucción para asegurarme de convertir toda la columna a ese formato da un error

#df1['release_date_new']  = pd.to_datetime(df1['release_date'], format = '%Y-%m-%d')
#ValueError: time data "1" doesn't match format "%Y-%m-%d", at position 10394. You might want to try:
#    - passing `format` if your strings have a consistent format;

# Debe haber algunas filas con otro formato. O algo diferente

In [13]:
# Reviso los 4 primeros caracteres de release_date para ver si todos estan en formato YYYY-
# df.assign(col2=df['col1'].str[0:3])
df['release_date'].str[0:4].unique()

array(['1995', '1996', '1994', '1997', '1976', '1992', '1967', '1993',
       '1964', '1977', '1965', '1982', '1985', '1990', '1991', '1989',
       '1937', '1940', '1969', '1981', '1974', '1970', '1960', '1955',
       '1959', '1968', '1980', '1988', '1975', '2002', '1948', '1943',
       '1950', '1987', '1973', '1956', '1958', '1949', '1972', '1953',
       '1998', '1933', '2010', '1952', '1951', '1957', '1961', '1954',
       '1934', '1944', '1963', '1942', '1941', '1939', '1947', '1946',
       '1945', '1938', '1935', '1936', '1926', '1932', '1979', '1971',
       '1986', '2013', '1978', '1966', '1962', '1983', '1984', '1931',
       '1922', '1999', '1927', '1929', '1930', '1928', '2012', '1925',
       '2000', '1919', '1923', '1920', '1918', '1921', '2001', '2011',
       '1924', '2003', '2004', '1915', '1916', '1917', '2005', '2006',
       '1902', '1903', '2007', '2008', '2009', '1914', '1912', '1913',
       '1898', '1899', '1894', '1909', '1910', '1901', '1893', '1896',
      

In [14]:
# Hay filas con str[0:4] '1' '12' '22'
# Miro qué datos hay en las otras columnas de esas filas para tratar de recuperar algo 
# df.loc[df['column_name'].isin('1' '12' '22')]
# df1.loc[df1['release_date'].str[0:4] == '1']
df.loc[df['release_date'].str[0:4].isin(['1', '12', '22'])]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return
19730,0.065736,0.0,"[{'name': 'Carousel Productions', 'id': 11176}...",1997-08-20,104.0,Released,NaN,False,6.0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
29503,1.931659,0.0,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",2012-09-29,68.0,Released,NaN,False,7.0,12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
35587,2.185485,0.0,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",2014-01-01,82.0,Released,Beware Of Frost Bites,False,4.3,22,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [15]:
# Los datos estan corridos. Son solo 3 filas.
# Borro las 3 filas que tienen valores invalidos en release_date
df.drop([19730, 29503,35587], inplace=True)

In [16]:
# Por las dudas, reviso también las posiciones correspondientes a los meses y los días
print('Deberían ser números válidos de meses:', df['release_date'].str[5:7].unique())
print('Deberían ser números válidos de días:', df['release_date'].str[8:10].unique())

Deberían ser números válidos de meses: ['10' '12' '02' '11' '09' '05' '04' '08' '07' '06' '01' '03']
Deberían ser números válidos de días: ['30' '15' '22' '10' '16' '17' '13' '09' '21' '20' '27' '06' '11' '29'
 '18' '08' '19' '14' '05' '01' '02' '12' '26' '03' '23' '31' '28' '07'
 '24' '25' '04']


In [17]:
# Ya eliminé las filas los valores inválidos. Y las posiciones de meses y dias tienen los valores esperados
# Ahora tendría que andar bien

# Use the .dt.year accessor to extract the year from a DateTime column in Pandas.
# Ensure the DateTime column is in the correct format using pd.to_datetime() if needed.

df['release_date']  = pd.to_datetime(df['release_date'], format = '%Y-%m-%d')

# Y confirmo que convirtió la columna a datetime
df['release_date'].dtype

dtype('<M8[ns]')

In [18]:
# crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.
# extraigo el año de la fecha de estreno
df['release_year'] = df['release_date'].dt.year

# Y reviso los años (re porfiada!)
df['release_year'].unique()

array([1995, 1996, 1994, 1997, 1976, 1992, 1967, 1993, 1964, 1977, 1965,
       1982, 1985, 1990, 1991, 1989, 1937, 1940, 1969, 1981, 1974, 1970,
       1960, 1955, 1959, 1968, 1980, 1988, 1975, 2002, 1948, 1943, 1950,
       1987, 1973, 1956, 1958, 1949, 1972, 1953, 1998, 1933, 2010, 1952,
       1951, 1957, 1961, 1954, 1934, 1944, 1963, 1942, 1941, 1939, 1947,
       1946, 1945, 1938, 1935, 1936, 1926, 1932, 1979, 1971, 1986, 2013,
       1978, 1966, 1962, 1983, 1984, 1931, 1922, 1999, 1927, 1929, 1930,
       1928, 2012, 1925, 2000, 1919, 1923, 1920, 1918, 1921, 2001, 2011,
       1924, 2003, 2004, 1915, 1916, 1917, 2005, 2006, 1902, 1903, 2007,
       2008, 2009, 1914, 1912, 1913, 1898, 1899, 1894, 1909, 1910, 1901,
       1893, 1896, 2014, 2016, 2015, 1895, 1911, 1900, 2020, 2017, 1905,
       1904, 1891, 2018, 1892, 1908, 1897, 1887, 1888, 1890, 1878, 1874,
       1906, 1883, 1907], dtype=int32)

In [21]:
# Guardo el dataset movies ya transformado para no tener que volver a correr todo. 
# Usos futuros (EDA y funciones)
df.to_csv("movies_ds_transform.csv" , index=False)

Trasnformación 6:</br>
Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [ ]:
# Analizo que columnas necesitaré para cada endpoint de la API (sugirieron que no transformáramos columnas que no necesitaremos)
# En otra notebook mostraré lo que preparé para cada función

'''
cantidad_filmaciones_mes: 
 - id de la película (ya me di cuenta que no necesito el título)
 - mes de estreno en español o número del mes (PREPARAR ESTA COLUMNA)

cantidad_filmaciones_dia: 
 - id de la película 
 - día de la semana en que se estrenó la película en idioma español (el día de la semana, no el estreno en español) (PREPARAR ESTA COLUMNA)

score_titulo: 
 - id de la película 
 - title
 - release_year
 - popularity (piden score/popularidad. Es esa columna)
   (ya tengo todas las columnas necesarias) 
 
votos_titulo: 
 - id de la película 
 - title,
 - release_year,
 - vote_average,
 - vote_count
   (tengo las columnas necesarias) 

get_actor: 
 - id de la película 
 - id del actor (campos id de la columna cast de la tabla credits) 
 - nombre del actor (campos name de la columna cast de la tabla credits)
 - título de cada pelicula en la qu participó
 - return de cada pelicula en la que participó
 - es_director (PREPARAR ESTA COLUMNA indicando los actores que tambien son directores porque hay que informar eso y no mostrar sus datos)
 NECESITO DESANIDAR LA COLUMNA CAST DE LA TABLA CREDITS PARA CONSEGUIR LOS DATOS DE LOS ACTORES. Hay varios actores por fila
 
get_director: 
 - id de la película 
 - id del director (id de la columna crew de la tabla credits donde 'job': 'Director')
 - nombre del director (name de la columna crew de la tabla credits donde 'job': 'Director') 
 - title
 - release_year
 - budget
 - revenue
 - return
 NECESITO DESANIDAR LA COLUMNA CREW DE LA TABLA CREDITS PARA CONSEGUIR LOS DATOS DE LOS DIRECTORES'''

Desanidaré los campos de las columnas cast y crew de la tabla credits porque necesito algunos valores para las funciones get_actor y get_director

In [ ]:
credits_aftermerge.to_csv("credits_sin_duplic.csv" , index=False)

Transformación 6.1: desanido la columna cast

In [19]:
# Leo el dataset credit y veo info 

# Nota: Al hacer el EDA encontré que habia ids duplicados de películas en los datasets de movies y de credits
# Tambien habia filas de películas y credits que no se correspondian entre si
# En la notebook de EDA elimine los duplicados e hice un inner join entre ambos datasets. Los guarde por separado
# El de movies sin duplicados lo usare en la notebook donde desarrollo las funciones
# El de credits lo leo aqui porque necesito dejar preparadas las columnas de actores y directores sin duplicados
dfc = pd.read_csv('credits_sin_duplic.csv')
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45345 non-null  int64 
 1   cast    45345 non-null  object
 2   crew    45345 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [20]:
# veo los datos que tienen las columnas cast y crew en una de sus filas
print('col cast:', dfc['cast'].iloc[0])
print('col crew:', dfc['crew'].iloc[0])

col cast: [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6

In [21]:
# veo como es la columna cast. dice type Object
dfc['cast'].unique()

array(["[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6Jq

In [ ]:
# Tiene comillas. Cada fila es un string. No son listas de diccionarios. Hay que desarmar la fila y convertirlo a eso
# Antes de desanidarlo

In [22]:
# armo una funcion para desanidar las columnas necesarias (con muchas ayudas varias)

def unnest_col(data, col_name, prefix_col):
    ''' Args: 
        - data: dataframe donde esta la columna a desanidar
        - col_name: nombre de la columna a desanidar
        - prefix_col: prefijo que tendrán cada una de las columnas generadas al desanidar la columna col_name 
    '''
    # cada fila es un string, no son listas porque tienen "" 
    # convierte la columna string a lista de diccionario
    data[col_name] = data[col_name].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) and pd.notnull(x) else [])
    
    # hace una fila por cada elemento de la lista de diccionarios
    unnested_data = data.explode(col_name).reset_index(drop= True)
    
    # arma una columna por cada elemento del diccionario, si no es nulo
    if unnested_data[col_name].notnull().any():
        unnested_keys = pd.json_normalize(unnested_data[col_name])
        unnested_keys.columns = [f"{prefix_col}_{col}" for col in unnested_keys.columns]
        unnested_data = pd.concat([unnested_data.drop(columns = [col_name]), unnested_keys], axis=1)
    return unnested_data

In [23]:
# desanido la columna cast del dataset credit para obtener los datos necesario para la funcion get_actor
# unnest_col(data, col_name, prefix_col)
dfc_cast = unnest_col(dfc, 'cast', 'cast')

In [24]:
# veo los datos de una de las filas para saber qué columnas necesito
dfc_cast.iloc[0]

id                                                                 862
crew                 [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
cast_cast_id                                                      14.0
cast_character                                           Woody (voice)
cast_credit_id                                52fe4284c3a36847f8024f95
cast_gender                                                        2.0
cast_id                                                           31.0
cast_name                                                    Tom Hanks
cast_order                                                         0.0
cast_profile_path                     /pQFoyx7rp09CJTAb932F2g8Nlho.jpg
Name: 0, dtype: object

In [25]:
# necesito las columnas id, cast_id y cast_name (estos dos ultimos son los nombres de cada uno de los actores que participan en una pelicula)
# el id de la pelicula esta repetido en muchas filas
# tendre que seleccionar solo algunas peliculas para la funcion get_actor porque si no el dataset sera muy pesado (se me ocurre)
# VER ESTO MAS ADELANTE

# selecciono las columnas que necesito y guardo el dataset para mas adelante. (Reservar, como dicen en las recetas)
# (Podria cambiar los nombres de cast a actor. Pero ya desarrollé las funciones asi que mejor no los modifico)

# Extracting specific selected columns to new DataFrame as a copy
# If you want to modify the new dataframe at all you'll probably want to use .copy() to avoid a SettingWithCopyWarning.

dfc_actors = dfc_cast[['id', 'cast_id' ,  'cast_name']].copy()
dfc_actors.to_csv("credits_actors.csv" , index=False)

Transformación 6.2: desanido la columna crew

In [26]:
# desanido la columna cast del dataset credit para obtener los datos necesario para la funcion get_actor
# unnest_col(data, col_name, prefix_col)
dfc_crew = unnest_col(dfc, 'crew', 'crew')

In [27]:
# Veo como quedo el dataset despues de desanidar la columna
dfc_crew.head(5)

,id,cast,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [28]:
# bastante feo
# Necesito las filas donde crew_job ='Director'

# selecciono esas filas y veo como queda
# rslt_df = dataframe.loc[dataframe['Percentage'] != 95]
dfc_crew_dir = dfc_crew.loc[dfc_crew['crew_job'] == 'Director']

dfc_crew_dir.head(5)

,id,cast,crew_credit_id,crew_department,crew_gender,crew_id,crew_job,crew_name,crew_profile_path
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
109,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",52fe44bfc3a36847f80a7c7d,Directing,2.0,4945.0,Director,Joe Johnston,/fok4jaO62v5IP6hkpaaAcXuw2H.jpg
122,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",52fe466a9251416c75077a89,Directing,2.0,26502.0,Director,Howard Deutch,/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg
126,31357,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...",52fe44779251416c91011acb,Directing,2.0,2178.0,Director,Forest Whitaker,/4pMQkelS5lK661m9Kz3oIxLYiyS.jpg
141,11862,"[{'cast_id': 1, 'character': 'George Banks', '...",52fe44959251416c75039eef,Directing,2.0,56106.0,Director,Charles Shyer,/hnWGd74CbmTcDCFQiJ8SYLazIXW.jpg


In [29]:
# selecciono las columnas que necesito
dfc_crew_dir1 = dfc_crew_dir[['id', 'crew_id', 'crew_name']]

# parece que 'crew_id' es float. 
dfc_crew_dir1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48957 entries, 0 to 464449
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         48957 non-null  int64  
 1   crew_id    48957 non-null  float64
 2   crew_name  48957 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB


In [30]:
# convierto id_crew a integer. (Por las dudas. Nunca vi ids tipo float)

# converting 'Weight' from float to int
# df['Weight'] = df['Weight'].astype(int)
dfc_crew_dir1['crew_id'] = dfc_crew_dir1['crew_id'].astype(int)

In [31]:
# Guardo el dataset con los datos de directores para usar en las funciones
dfc_crew_dir1.to_csv("credits_directors.csv" , index=False)

Me parece que ya hice todas las transformaciones pedidas (espero que así sea)